# 自我发现

一个实现了[自我发现论文](https://arxiv.org/pdf/2402.03620.pdf)的项目。

基于[@catid的这个实现](https://github.com/catid/self-discover/tree/main?tab=readme-ov-file)。

In [1]:
from langchain_openai import ChatOpenAI

In [2]:

# 创建ChatOpenAI对象，并设置参数temperature为0，model为"gpt-4-turbo-preview"
model = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview")

In [3]:
# 导入所需模块
from langchain import hub
from langchain_core.prompts import PromptTemplate

In [4]:
# 从hub中拉取"hwchase17/self-discovery-select"模块
select_prompt = hub.pull("hwchase17/self-discovery-select")

In [5]:
# 调用select_prompt对象的pretty_print()方法
select_prompt.pretty_print()

Select several reasoning modules that are crucial to utilize in order to solve the given task:

All reasoning module descriptions:
{reasoning_modules}

Task: {task_description}

Select several modules are crucial for solving the task above:



In [6]:
# 导入所需的库
import hub

# 从hub库中拉取"hwchase17/self-discovery-adapt"模型
adapt_prompt = hub.pull("hwchase17/self-discovery-adapt")

In [7]:
# 打印 adapt_prompt 对象的信息
adapt_prompt.pretty_print()

Rephrase and specify each reasoning module so that it better helps solving the task:

SELECTED module descriptions:
{selected_modules}

Task: {task_description}

Adapt each reasoning module description to better solve the task:



In [8]:
# 从hub中拉取名为"hwchase17/self-discovery-structure"的模型
structured_prompt = hub.pull("hwchase17/self-discovery-structure")

In [9]:
# 打印 structured_prompt 对象的漂亮格式
structured_prompt.pretty_print()

Operationalize the reasoning modules into a step-by-step reasoning plan in JSON format:

Here's an example:

Example task:

If you follow these instructions, do you return to the starting point? Always face forward. Take 1 step backward. Take 9 steps left. Take 2 steps backward. Take 6 steps forward. Take 4 steps forward. Take 4 steps backward. Take 3 steps right.

Example reasoning structure:

{
    "Position after instruction 1":
    "Position after instruction 2":
    "Position after instruction n":
    "Is final position the same as starting position":
}

Adapted module description:
{adapted_modules}

Task: {task_description}

Implement a reasoning structure for solvers to follow step-by-step and arrive at correct answer.

Note: do NOT actually arrive at a conclusion in this pass. Your job is to generate a PLAN so that in the future you can fill it out and arrive at the correct conclusion for tasks like this


In [10]:
# 从hub中获取模型
reasoning_prompt = hub.pull("hwchase17/self-discovery-reasoning")


In [11]:
# 打印 reasoning_prompt 对象的内容
reasoning_prompt.pretty_print()

Follow the step-by-step reasoning plan in JSON to correctly solve the task. Fill in the values following the keys by reasoning specifically about the task given. Do not simply rephrase the keys.
    
Reasoning Structure:
{reasoning_structure}

Task: {task_description}


In [12]:
reasoning_prompt

PromptTemplate(input_variables=['reasoning_structure', 'task_description'], template='Follow the step-by-step reasoning plan in JSON to correctly solve the task. Fill in the values following the keys by reasoning specifically about the task given. Do not simply rephrase the keys.\n    \nReasoning Structure:\n{reasoning_structure}\n\nTask: {task_description}')

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [14]:
# 创建一个select_chain对象，该对象由select_prompt、model和StrOutputParser()组成
select_chain = select_prompt | model | StrOutputParser()

In [15]:
# 创建一个适应链对象
adapt_chain = adapt_prompt | model | StrOutputParser()

# adapt_prompt: 适应提示器对象，用于根据输入调整模型的行为
# model: 模型对象，用于生成输出
# StrOutputParser: 字符串输出解析器对象，用于解析模型生成的字符串输出

In [16]:
# 创建一个结构链对象
structure_chain = structured_prompt | model | StrOutputParser()

# structured_prompt 是一个结构化的输入提示对象
# model 是一个模型对象，用于生成输出
# StrOutputParser 是一个字符串输出解析器对象，用于解析模型生成的字符串输出

In [17]:
# 创建一个推理链，包括推理提示、模型和字符串输出解析器
reasoning_chain = reasoning_prompt | model | StrOutputParser()

In [18]:
# 创建一个链式操作，依次对 select_chain、adapt_chain、structure_chain 和 reasoning_chain 进行操作，并将结果分别赋值给 selected_modules、adapted_modules、reasoning_structure 和 answer
overall_chain = (
    RunnablePassthrough.assign(selected_modules=select_chain)
    .assign(adapted_modules=adapt_chain)
    .assign(reasoning_structure=structure_chain)
    .assign(answer=reasoning_chain)
)

In [19]:
# 定义推理模块列表
reasoning_modules = [
    "1. 如何设计一个实验来帮助解决这个问题？",
    "2. 列出解决这个问题的想法，并逐一应用它们来看是否能取得任何进展。",
    # "3. 如何衡量这个问题的进展？",
    "4. 如何简化这个问题，使其更容易解决？",
    "5. 这个问题的关键假设是什么？",
    "6. 每个解决方案的潜在风险和缺点是什么？",
    "7. 这个问题的替代观点或观点是什么？",
    "8. 这个问题及其解决方案的长期影响是什么？",
    "9. 如何将这个问题分解成更小、更易管理的部分？",
    "10. 批判性思维：这种风格涉及从不同的角度分析问题，质疑假设，并评估可用的证据或信息。它侧重于逻辑推理、基于证据的决策和识别潜在的偏见或思维缺陷。",
    "11. 尝试创造性思维，产生创新和超越传统边界的想法来解决问题。探索非传统的解决方案，超越传统边界，鼓励想象力和独创性。",
    # "12. 寻求他人的意见和合作来解决问题。强调团队合作、开放沟通，并利用团队的多样化观点和专业知识，提出有效的解决方案。",
    "13. 使用系统思维：将问题视为更大系统的一部分，并理解各种元素之间的相互关系。侧重于识别影响问题的根本原因、反馈环路和相互依赖关系，并制定全面解决方案，解决整个系统的问题。",
    "14. 使用风险分析：评估与不同解决方案或方法相关的潜在风险、不确定性和权衡。强调评估潜在后果和成功或失败的可能性，并根据风险和收益的平衡分析做出明智的决策。",
    # "15. 使用反思性思维：从问题中退后，花时间进行内省和自我反思。检查个人偏见、假设和可能影响问题解决的心智模式，并愿意从过去的经验中学习，以改进未来的方法。",
    "16. 需要解决的核心问题是什么？",
    "17. 导致问题的潜在原因或因素是什么？",
    "18. 以前是否尝试过任何潜在的解决方案或策略？如果是，结果和经验教训是什么？",
    "19. 解决这个问题可能出现的潜在障碍或挑战是什么？",
    "20. 是否有任何相关的数据或信息可以提供对问题的见解？如果有，有哪些数据来源，以及如何分析这些数据？",
    "21. 是否有任何利益相关者或直接受到问题影响的个人？他们的观点和需求是什么？",
    "22. 解决问题所需的资源（财务、人力、技术等）是什么？",
    "23. 如何衡量或评估解决问题的进展或成功？",
    "24. 可以使用哪些指标或度量标准？",
    "25. 问题是需要特定专业知识或技能的技术性或实际问题吗？还是更多的是概念性或理论性问题？",
    "26. 问题是否涉及物理约束，如有限的资源、基础设施或空间？",
    "27. 问题是否涉及人类行为，如社会、文化或心理问题？",
    "28. 问题是否涉及决策或规划，需要在不确定性或具有竞争目标的情况下做出选择？",
    "29. 问题是否是需要数据分析、建模或优化技术的分析性问题？",
    "30. 问题是否是需要创造性解决方案和创新的设计挑战？",
    "31. 问题是否需要解决系统性或结构性问题，而不仅仅是个别情况？",
    "32. 问题是否时间紧迫或紧急，需要立即关注和行动？",
    "33. 这种问题规范通常产生哪些解决方案？",
    "34. 鉴于问题规范和当前最佳解决方案，对其他可能的解决方案有何猜测。",
    "35. 让我们想象当前最佳解决方案完全错误，还有其他思考问题规范的方式吗？",
    "36. 根据你对这种问题规范的了解，修改当前最佳解决方案的最佳方法是什么？",
    "37. 忽略当前最佳解决方案，为问题创建一个全新的解决方案。",
    # "38. 让我们一步一步地思考。",
    "39. 让我们制定一个逐步计划，并用良好的符号和解释来实施它。",
]

# 任务示例
task_example = "Lisa有10个苹果。她给了她的朋友3个苹果，然后从商店买了5个苹果。现在Lisa有多少个苹果？"

task_example = """这个SVG路径元素 <path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L
45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/> 画出了一个：
(A) 圆 (B) 七边形 (C) 六边形 (D) 风筝 (E) 线 (F) 八边形 (G) 五边形 (H) 矩形 (I) 扇形 (J) 三角形"""

In [20]:
# 将 reasoning_modules 列表中的元素用换行符连接成一个字符串
reasoning_modules_str = "\n".join(reasoning_modules)

In [65]:
# 调用 overall_chain 的 invoke 方法，传入一个包含任务描述和推理模块的字典作为参数
overall_chain.invoke(
    {"task_description": task_example, "reasoning_modules": reasoning_modules_str}
)

{'task_description': 'This SVG path element <path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L\n45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/> draws a:\n(A) circle (B) heptagon (C) hexagon (D) kite (E) line (F) octagon (G) pentagon(H) rectangle (I) sector (J) triangle',
 'reasoning_modules': '1. How could I devise an experiment to help solve that problem?\n2. Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.\n4. How can I simplify the problem so that it is easier to solve?\n5. What are the key assumptions underlying this problem?\n6. What are the potential risks and drawbacks of each solution?\n7. What are the alternative perspectives or viewpoints on this problem?\n8. What are the long-term implications of this problem and its solutions?\n9. How can I break down this problem into smaller, more manageable parts?\n10. Critical Thinking: This style